## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users/Jeremy/Desktop/SKOOL/DA_Bootcamp/Python Module/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Ionia,42.9873,-85.0711,91.60,46,0,12.66,US,2022-06-15 23:03:47,clear sky
1,1,Samarai,-10.6167,150.6667,82.63,79,75,10.40,PG,2022-06-15 23:03:47,broken clouds
2,2,Cusuna,15.8500,-85.2333,81.25,73,99,8.32,HN,2022-06-15 23:03:47,overcast clouds
3,3,Krasnoarmeysk,51.0239,45.6969,62.51,82,94,7.99,RU,2022-06-15 23:03:48,overcast clouds
4,4,Pevek,69.7008,170.3133,35.11,78,6,5.73,RU,2022-06-15 23:03:48,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
1,1,Samarai,-10.6167,150.6667,82.63,79,75,10.40,PG,2022-06-15 23:03:47,broken clouds
2,2,Cusuna,15.8500,-85.2333,81.25,73,99,8.32,HN,2022-06-15 23:03:47,overcast clouds
5,5,Vaini,-21.2000,-175.2000,80.76,74,75,6.91,TO,2022-06-15 23:03:48,broken clouds
7,7,Makakilo City,21.3469,-158.0858,83.01,63,75,12.66,US,2022-06-15 22:58:58,broken clouds
17,17,Hilo,19.7297,-155.0900,82.27,73,40,10.36,US,2022-06-15 23:03:53,scattered clouds
21,21,Puerto Escondido,15.8500,-97.0667,79.63,61,100,9.22,MX,2022-06-15 23:03:54,overcast clouds
25,25,Albany,42.6001,-73.9662,78.67,53,86,2.57,US,2022-06-15 23:01:43,overcast clouds
35,35,Dingle,10.9995,122.6711,80.67,83,69,4.00,PH,2022-06-15 23:03:58,broken clouds
38,38,Luba,3.4568,8.5547,78.60,86,63,10.31,GQ,2022-06-15 23:03:59,light rain
39,39,Roma,43.2128,-75.4557,84.34,45,0,5.75,US,2022-06-15 23:01:38,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Unnamed: 0             0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# no null rows, preferred_cities_df is clean

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Samarai,PG,82.63,broken clouds,-10.6167,150.6667,
2,Cusuna,HN,81.25,overcast clouds,15.8500,-85.2333,
5,Vaini,TO,80.76,broken clouds,-21.2000,-175.2000,
7,Makakilo City,US,83.01,broken clouds,21.3469,-158.0858,
17,Hilo,US,82.27,scattered clouds,19.7297,-155.0900,
21,Puerto Escondido,MX,79.63,overcast clouds,15.8500,-97.0667,
25,Albany,US,78.67,overcast clouds,42.6001,-73.9662,
35,Dingle,PH,80.67,broken clouds,10.9995,122.6711,
38,Luba,GQ,78.60,light rain,3.4568,8.5547,
39,Roma,US,84.34,clear sky,43.2128,-75.4557,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=['Hotel Name'])
clean_hotel_df.count()

City                   196
Country                196
Max Temp               196
Current Description    196
Lat                    196
Lng                    196
Hotel Name             196
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "./WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
<
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations,  info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))